In [1]:
import os
import numpy as np
from datetime import datetime

import faster_rcnn.roi_data_layer.roidb as rdl_roidb
from faster_rcnn.datasets.factory import get_imdb
from faster_rcnn.fast_rcnn.config import cfg, cfg_from_file

def log_print(text, color=None, on_color=None, attrs=None):
    if cprint is not None:
        cprint(text, color=color, on_color=on_color, attrs=attrs)
    else:
        print(text)



# hyper-parameters
# ------------
imdb_name = 'sunrgbd_train_rgb_i_100_8bits' #train
cfg_file = 'experiments/cfgs/faster_rcnn_end2end_sunrgbd.yml'
pretrained_model = 'data/pretrained_model/VGG_imagenet.npy'
output_dir = 'models/sunrgbd_rgb100/'




# load config
cfg_from_file(cfg_file)

# load data
imdb = get_imdb(imdb_name)

<bound method sunrgbd.default_roidb of <faster_rcnn.datasets.sunrgbd.sunrgbd object at 0x7f442b6a8910>>


In [2]:
gt_roidb=imdb.gt_roidb()

sunrgbd_train_rgb_i_100_8bits gt roidb loaded from /c16/THESE.JORIS/datasets/SUNRGBD_pv/faster_rcnn/cache/sunrgbd_train_rgb_i_100_8bits_gt_roidb.pkl


In [3]:
# print "analysing..."
myClassesDict = {}
for img_rois in gt_roidb:
    for id_cls in img_rois['gt_classes']:
        myClassesDict[imdb._classes[id_cls]] = myClassesDict.get(imdb._classes[id_cls], 0) + 1
        
a = sorted(myClassesDict, key=myClassesDict.get)
for cls in a:
    print cls, myClassesDict[cls]
# print myClassesDict
# print "analysed !"

shelves 1
shower_curtain 11
window 32
bathtub 134
fridge 183
toilet 205
curtain 230
towel 262
bag 269
blinds 326
tv 357
mirror 446
sink 530
bookshelf 703
night_stand 869
paper 1280
books 1397
bed 1512
box 1595
picture 1765
dresser 1862
door 2068
counter 2109
person 2411
whiteboard 2461
sofa 2687
cabinet 3046
lamp 4639
desk 5474
pillow 8988
table 15706
chair 145894


In [4]:
myFreqDict = {}
total = sum(myClassesDict.values())
for cls in myClassesDict.keys():
    myFreqDict[cls]=round(float(myClassesDict[cls])/total,5) + 1e-05
        
b = sorted(myFreqDict, key=myFreqDict.get)
ind_med_freq = (len(b)-1)/2
medianFreq = myFreqDict[b[ind_med_freq]]
for cls in b:
    print cls, myFreqDict[cls]

# myFreqDict

shelves 1e-05
shower_curtain 6e-05
window 0.00016
bathtub 0.00065
fridge 0.00088
toilet 0.00099
curtain 0.00111
towel 0.00126
bag 0.00129
blinds 0.00157
tv 0.00171
mirror 0.00214
sink 0.00254
bookshelf 0.00337
night_stand 0.00416
paper 0.00612
books 0.00668
bed 0.00723
box 0.00763
picture 0.00844
dresser 0.0089
door 0.00988
counter 0.01008
person 0.01152
whiteboard 0.01176
sofa 0.01284
cabinet 0.01455
lamp 0.02216
desk 0.02614
pillow 0.04292
table 0.075
chair 0.69656


In [13]:
myWeightsDict = {}
for cls in myClassesDict.keys():
    #myWeightsDict[cls] = 1+round(medianFreq/myFreqDict[cls],3)
    myWeightsDict[cls] = float(total-myClassesDict[cls])/(100*myClassesDict[cls])
    print 'classes_weights_dict[\''+cls+'\'] = '+str(myWeightsDict[cls])

classes_weights_dict['toilet'] = 10.2071707317
classes_weights_dict['whiteboard'] = 0.841084924827
classes_weights_dict['night_stand'] = 2.40026467204
classes_weights_dict['paper'] = 1.62634375
classes_weights_dict['mirror'] = 4.68623318386
classes_weights_dict['table'] = 0.123357952375
classes_weights_dict['chair'] = 0.0043564505737
classes_weights_dict['bag'] = 7.7763197026
classes_weights_dict['tv'] = 5.85700280112
classes_weights_dict['window'] = 65.44375
classes_weights_dict['lamp'] = 0.441502478983
classes_weights_dict['curtain'] = 9.09660869565
classes_weights_dict['blinds'] = 6.41490797546
classes_weights_dict['dresser'] = 1.11487647691
classes_weights_dict['picture'] = 1.17669688385
classes_weights_dict['door'] = 1.00282398453
classes_weights_dict['shower_curtain'] = 190.400909091
classes_weights_dict['fridge'] = 11.4354644809
classes_weights_dict['cabinet'] = 0.677629678267
classes_weights_dict['sink'] = 3.9419245283
classes_weights_dict['desk'] = 0.372630617464
classes_weigh

In [14]:
_classes = ('__background__', # always index 0
                         #'wall',
                        #'floor',
                        'cabinet',
                        'bed',
                        'chair',
                        'sofa',
                        'table',
                        'door',
                        'window',
                        'bookshelf',
                        'picture',
                        'counter',
                        'blinds',
                        'desk',
                        'shelves',
                        'curtain',
                        'dresser',
                        'pillow',
                        'mirror',
                        #'floor_mat',
                        #'clothes',
                        #'ceiling',
                        'books',
                        'fridge',
                        'tv',
                        'paper',
                        'towel',
                        'shower_curtain',
                        'box',
                        'whiteboard',
                        'person',
                        'night_stand',
                        'toilet',
                        'sink',
                        'lamp',
                        'bathtub',
                        'bag')
_class_to_ind = dict(zip(_classes, xrange(len(_classes))))

In [15]:
weights = [-1]
for i in range(1,len(_classes)):
    print _classes[i],myWeightsDict[_classes[i]]
    weights= weights+[myWeightsDict[_classes[i]]]
print weights

cabinet 0.677629678267
bed 1.37526455026
chair 0.0043564505737
sofa 0.769501302568
table 0.123357952375
door 1.00282398453
window 65.44375
bookshelf 2.96940256046
picture 1.17669688385
counter 0.983134186818
blinds 6.41490797546
desk 0.372630617464
shelves 2094.51
curtain 9.09660869565
dresser 1.11487647691
pillow 0.223035157988
mirror 4.68623318386
books 1.48929849678
fridge 11.4354644809
tv 5.85700280112
paper 1.62634375
towel 7.98435114504
shower_curtain 190.400909091
box 1.30317868339
whiteboard 0.841084924827
person 0.858734964745
night_stand 2.40026467204
toilet 10.2071707317
sink 3.9419245283
lamp 0.441502478983
bathtub 15.6207462687
bag 7.7763197026
[-1, 0.6776296782665792, 1.3752645502645502, 0.0043564505737041966, 0.7695013025679196, 0.12335795237488857, 1.0028239845261122, 65.44375, 2.969402560455192, 1.1766968838526912, 0.9831341868183974, 6.414907975460123, 0.37263061746437703, 2094.51, 9.096608695652174, 1.114876476906552, 0.22303515798842902, 4.686233183856502, 1.4892984

In [16]:
len(_classes)

33

In [17]:
len(weights)

33